# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f24bc531eb0>
├── 'a' --> tensor([[ 0.6932, -1.0206,  0.3206],
│                   [ 0.7897,  0.1470, -0.4013]])
└── 'x' --> <FastTreeValue 0x7f24c05bb940>
    └── 'c' --> tensor([[ 1.8835, -0.4989, -1.1123, -0.2295],
                        [ 0.1457, -1.1854,  0.3117,  0.8347],
                        [-0.3989, -0.2347, -2.7108, -0.1627]])

In [4]:
t.a

tensor([[ 0.6932, -1.0206,  0.3206],
        [ 0.7897,  0.1470, -0.4013]])

In [5]:
%timeit t.a

67.4 ns ± 0.167 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f24bc531eb0>
├── 'a' --> tensor([[-1.2573,  1.8381, -0.1722],
│                   [ 0.5867, -1.6721,  0.8087]])
└── 'x' --> <FastTreeValue 0x7f24c05bb940>
    └── 'c' --> tensor([[ 1.8835, -0.4989, -1.1123, -0.2295],
                        [ 0.1457, -1.1854,  0.3117,  0.8347],
                        [-0.3989, -0.2347, -2.7108, -0.1627]])

In [7]:
%timeit t.a = new_value

68.4 ns ± 0.0193 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.6932, -1.0206,  0.3206],
               [ 0.7897,  0.1470, -0.4013]]),
    x: Batch(
           c: tensor([[ 1.8835, -0.4989, -1.1123, -0.2295],
                      [ 0.1457, -1.1854,  0.3117,  0.8347],
                      [-0.3989, -0.2347, -2.7108, -0.1627]]),
       ),
)

In [10]:
b.a

tensor([[ 0.6932, -1.0206,  0.3206],
        [ 0.7897,  0.1470, -0.4013]])

In [11]:
%timeit b.a

58.4 ns ± 0.0705 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.4049, -1.3676, -0.1918],
               [ 0.6414, -0.2744, -0.6001]]),
    x: Batch(
           c: tensor([[ 1.8835, -0.4989, -1.1123, -0.2295],
                      [ 0.1457, -1.1854,  0.3117,  0.8347],
                      [-0.3989, -0.2347, -2.7108, -0.1627]]),
       ),
)

In [13]:
%timeit b.a = new_value

499 ns ± 0.185 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

861 ns ± 1.76 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.5 µs ± 15.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

145 µs ± 1.18 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

146 µs ± 594 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f23ef7fce20>
├── 'a' --> tensor([[[ 0.6932, -1.0206,  0.3206],
│                    [ 0.7897,  0.1470, -0.4013]],
│           
│                   [[ 0.6932, -1.0206,  0.3206],
│                    [ 0.7897,  0.1470, -0.4013]],
│           
│                   [[ 0.6932, -1.0206,  0.3206],
│                    [ 0.7897,  0.1470, -0.4013]],
│           
│                   [[ 0.6932, -1.0206,  0.3206],
│                    [ 0.7897,  0.1470, -0.4013]],
│           
│                   [[ 0.6932, -1.0206,  0.3206],
│                    [ 0.7897,  0.1470, -0.4013]],
│           
│                   [[ 0.6932, -1.0206,  0.3206],
│                    [ 0.7897,  0.1470, -0.4013]],
│           
│                   [[ 0.6932, -1.0206,  0.3206],
│                    [ 0.7897,  0.1470, -0.4013]],
│           
│                   [[ 0.6932, -1.0206,  0.3206],
│                    [ 0.7897,  0.1470, -0.4013]]])
└── 'x' --> <FastTreeValue 0x7f23f08caa30>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

33.2 µs ± 135 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f24bc516850>
├── 'a' --> tensor([[ 0.6932, -1.0206,  0.3206],
│                   [ 0.7897,  0.1470, -0.4013],
│                   [ 0.6932, -1.0206,  0.3206],
│                   [ 0.7897,  0.1470, -0.4013],
│                   [ 0.6932, -1.0206,  0.3206],
│                   [ 0.7897,  0.1470, -0.4013],
│                   [ 0.6932, -1.0206,  0.3206],
│                   [ 0.7897,  0.1470, -0.4013],
│                   [ 0.6932, -1.0206,  0.3206],
│                   [ 0.7897,  0.1470, -0.4013],
│                   [ 0.6932, -1.0206,  0.3206],
│                   [ 0.7897,  0.1470, -0.4013],
│                   [ 0.6932, -1.0206,  0.3206],
│                   [ 0.7897,  0.1470, -0.4013],
│                   [ 0.6932, -1.0206,  0.3206],
│                   [ 0.7897,  0.1470, -0.4013]])
└── 'x' --> <FastTreeValue 0x7f23f8a9c430>
    └── 'c' --> tensor([[ 1.8835, -0.4989, -1.1123, -0.2295],
                        [ 0.1457, -1.1854,  0.3117,  0.8347],
                 

In [23]:
%timeit t_cat(trees)

30.7 µs ± 35.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

64.6 µs ± 25.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.6932, -1.0206,  0.3206],
                [ 0.7897,  0.1470, -0.4013]],
       
               [[ 0.6932, -1.0206,  0.3206],
                [ 0.7897,  0.1470, -0.4013]],
       
               [[ 0.6932, -1.0206,  0.3206],
                [ 0.7897,  0.1470, -0.4013]],
       
               [[ 0.6932, -1.0206,  0.3206],
                [ 0.7897,  0.1470, -0.4013]],
       
               [[ 0.6932, -1.0206,  0.3206],
                [ 0.7897,  0.1470, -0.4013]],
       
               [[ 0.6932, -1.0206,  0.3206],
                [ 0.7897,  0.1470, -0.4013]],
       
               [[ 0.6932, -1.0206,  0.3206],
                [ 0.7897,  0.1470, -0.4013]],
       
               [[ 0.6932, -1.0206,  0.3206],
                [ 0.7897,  0.1470, -0.4013]]]),
    x: Batch(
           c: tensor([[[ 1.8835, -0.4989, -1.1123, -0.2295],
                       [ 0.1457, -1.1854,  0.3117,  0.8347],
                       [-0.3989, -0.2347, -2.7108, -0.1627]],
         

In [26]:
%timeit Batch.stack(batches)

78.6 µs ± 145 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.6932, -1.0206,  0.3206],
               [ 0.7897,  0.1470, -0.4013],
               [ 0.6932, -1.0206,  0.3206],
               [ 0.7897,  0.1470, -0.4013],
               [ 0.6932, -1.0206,  0.3206],
               [ 0.7897,  0.1470, -0.4013],
               [ 0.6932, -1.0206,  0.3206],
               [ 0.7897,  0.1470, -0.4013],
               [ 0.6932, -1.0206,  0.3206],
               [ 0.7897,  0.1470, -0.4013],
               [ 0.6932, -1.0206,  0.3206],
               [ 0.7897,  0.1470, -0.4013],
               [ 0.6932, -1.0206,  0.3206],
               [ 0.7897,  0.1470, -0.4013],
               [ 0.6932, -1.0206,  0.3206],
               [ 0.7897,  0.1470, -0.4013]]),
    x: Batch(
           c: tensor([[ 1.8835, -0.4989, -1.1123, -0.2295],
                      [ 0.1457, -1.1854,  0.3117,  0.8347],
                      [-0.3989, -0.2347, -2.7108, -0.1627],
                      [ 1.8835, -0.4989, -1.1123, -0.2295],
                      [ 0.1457, -

In [28]:
%timeit Batch.cat(batches)

149 µs ± 246 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

343 µs ± 5.85 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
